In [5]:
!streamlit run main_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.66:8501


  A new version of Streamlit is available.

  See what's new at https://discuss.streamlit.io/c/announcements

  Enter the following command to upgrade:
  $ pip install streamlit --upgrade

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bernardonobrepalma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bernardonobrepalma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
import os
import pandas as pd
import pdfplumber
from sentence_transformers import SentenceTransformer, util
import torch
import nltk
from nltk.corpus import stopwords
import re
import spacy
import streamlit as st
from io import BytesIO
import time
from keybert import KeyBERT
from openai import OpenAI
from jobs_scrape import scrape_my_jobs

/opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [17]:
# Download required NLTK data
try:
    nltk.download('stopwords')
    nltk.download('wordnet')
except Exception as e:
    st.warning(f"Error downloading NLTK data: {e}")

# Initialize Sentence-BERT model
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Initialize KeyBERT model
kw_model = KeyBERT(model='paraphrase-distilroberta-base-v1')

# Initialize stop words
try:
    stop_words = set(stopwords.words('english'))
except Exception as e:
    st.warning(f"Error loading stop words: {e}")
    stop_words = set()

# Load pre-trained SpaCy model for NER
nlp = spacy.load('en_core_web_sm')

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


In [18]:
import spacy

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

def classify_job_description(text):
    # Initialize sections
    sections = {'requirements': [], 'responsibilities': [], 'rest': []}
    current_section = 'rest'
    
    # Union of all keywords for requirements and responsibilities
    keywords_requirements_union = [
        "education and experience required", "knowledge and skills", "additional skills", 
        "bachelor’s degree", "master’s degree", "knowledge of programming languages", 
        "conceptual understanding of", "passion for new and emerging", "willingness to participate",
        "communicate effectively", "problem-solving skills", "able to produce", "teamwork skills", 
        "autonomy", "active learning", "active listening", "client expectations management",
        "at least 2 years of experience", "experience as a business analysis", "customer experience design roles", 
        "established quality standards", "industry benchmarks", "creating detailed and thorough as-is analysis", 
        "designing to-be processes", "understand the business requirements", "translate them into technical requirements", 
        "knowledge and experience of implementing", "proven experience of ba & change management", 
        "knowledge of agile methodology", "experience in alm tools", "experience with contact centre solutions", 
        "strong written and verbal communication skills", "certifications", "understanding of financial institutions",
        "experience as a business analyst", "experience with process modeling tools", "knowledge of jira and/or confluence tools",
        "fluency in microsoft office suite", "fluent in english", "Minimum 2 years ", "Minimum 3 years ", "Minimum 4 years ", 
        "Minimum 5 years ", "Minimum 6 years ", "Minimum 7 years ", "Minimum 8 years ", "Minimum 9 years ", "Minimum 10 years ",
        "years of experience", "experience in a similar role", "experience in a similar position", "experience in a similar environment",
        "looking for", "How about you?", "What we're looking for", "What we are looking for", "What we are looking for in you","what we are looking for in you",
        "what you'll bring", "what you will bring", "what you'll need", "what you will need", "what you'll do", "what you will do",
        "about you","Requirements", "requirements", "qualifications", "Qualifications", "skills", "Skills", "experience", "Experience","ability",
        "Ability","Knowledge", "knowledge", "Knowledgeable", "knowledgeable","diploma", "Academic degree", "academic degree", "certification", "Certification", 
        "certified", "Certified","Expertise", "expertise", "Expert", "expert", "Proficient", "proficient", "Competent", "competent", "Fluent", "fluent","Post-Bologna", 
        "post-Bologna", "Post Bologna", "post Bologna", "Post-Bologna degree", "post-Bologna degree", "Post Bologna degree", "post Bologna degree", "Master", "master",
        "Up to", "up to", "At least", "at least", "Minimum", "minimum", "Preferred", "preferred", "Plus", "plus", "Nice to have", "nice to have", "Desirable", "desirable",
        "have a Bachelor", "have a Master", "have a PhD", "have a Post-Bologna", "have a Post Bologna", "have a Post-Bologna degree", "have a Post Bologna degree"," have a Master's in a field",
        "have a Bachelor's in a field", "have a PhD in a field", "have a Post-Bologna in a field", "have a Post Bologna in a field", "have a Post-Bologna degree in a field",
        "have a bachelor's and a master's degree", "have a bachelor's and a PhD", "have a bachelor's and a Post-Bologna", "have a bachelor's and a Post Bologna",
        "what will make you succeed", "what will make you successful", "what will make you triumph", "what will make you win", "what will make you stand out",
        "expertise in", "Expertise in", "experience with", "Experience with", "knowledge of", "Knowledge of", "skills in", "Skills in", "ability to", "Ability to",
        "Good knowledge of", "good knowledge of", "Strong knowledge of", "strong knowledge of", "Excellent knowledge of", "excellent knowledge of", "Solid knowledge of", "solid knowledge of"
    ]
    keywords_responsibilities_union = [
        "responsible for delivery", "work under supervision", "integrate technical knowledge", 
        "apply company solutions", "provide technical consulting", "resolve technical issues", 
        "manage smaller projects/programs", "act professionally as trusted advisor", 
        "participates as part of a team", "understand the company's strategy", "keep refreshing technical knowledge", 
        "participate in technical or professional community events",
        "leading transformation and bau activities", "join our dynamic team", "recognising business requirements", 
        "analysing user stories", "managing the backlog", "engaging with project team members and stakeholders", 
        "ensure business requirements are captured", "ensuring timely and consistent communication", 
        "remain up to date with new practices",
        "responsible for working with company data", "reporting metrics", "analyzing methodologies", 
        "suggesting operation improvements", "building proposal evaluations", "in a cross-functional environment",
        "Develop", "develop", "Design", "design", "Create", "create","responsible for", "Responsible for", "responsible of", "Responsible of",
        "Process", "process", "Manage", "manage", "Lead", "lead", "Coordinate", "coordinate", "Collaborate", "collaborate", "Support", "support",
        "Implement", "implement", "Monitor", "monitor", "Analyze", "analyze", "Evaluate", "evaluate", "Define", "define", "Optimize", "optimize",
        "Ensure", "ensure", "Establish", "establish", "Maintain", "maintain", "Perform", "perform", "Participate", "participate", "Contribute", "contribute",
        "collect", "Collect", "prepare", "Prepare", "deliver", "Deliver", "report", "Report", "communicate", "Communicate", "engage", "Engage", "work", "Work",
        "collaborate", "Collaborate", "coordinate", "Coordinate", "support", "Support", "monitor", "Monitor", "analyze", "Analyze", "evaluate", "Evaluate",
        "report", "Report", "ensure", "Ensure", "maintain", "Maintain", "participate", "Participate", "contribute", "Contribute", "develop", "Develop",
        "identify","Identify" 
    ]
    
    # Process the document with spaCy
    doc = nlp(text)
    
    # Iterate over sentences to classify them based on keywords
    for sent in doc.sents:
        sentence = sent.text.strip().lower()
        if any(kw in sentence for kw in keywords_responsibilities_union):
            current_section = 'responsibilities'
        elif any(kw in sentence for kw in keywords_requirements_union):
            current_section = 'requirements'
        else:
            current_section = 'rest'
        
        sections[current_section].append(sent.text.strip())

    # Join sentences for each section
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Example job description text
job_description_example = """

About the job
This role has been designed as ‘Hybrid’ with an expectation that you will work on average 2-3 days per week from an HPE office.

Who We Are

Hewlett Packard Enterprise is the global edge-to-cloud company advancing the way people live and work. We help companies connect, protect, analyze, and act on their data and applications wherever they live, from edge to cloud, so they can turn insights into outcomes at the speed required to thrive in today’s complex world. Our culture thrives on finding new and better ways to accelerate what’s next. We know diverse backgrounds are valued and succeed here. We have the flexibility to manage our work and personal needs. We make bold moves, together, and are a force for good. If you are looking to stretch and grow your career our culture will embrace you. Open up opportunities with HPE.

Job Description

The AI & Data Junior Consultant is a customer facing role to provide AI & Data technology consulting to external customers and internal project teams. Responsible for providing technical contribution and/or leadership in the creation and delivery of technology solutions designed to meet customers’ business needs and drive outcomes for the customers.

The AI & Data Junior Consultant will keep pace with the advancement of leading-edge AI & Data technologies and industry/market trend while actively contributing to the company’s solutions portfolio through information and knowledge sharing ranging from technical expertise to methodologies based on experience gained from customer projects. Help to shape technical direction and technical strategies within the organization and for external customers.

The AI & Data Junior Consultant is accountable for contributing to organization’s profitability by generating and cultivating new business opportunities and by providing technical support for deal proposal development. The AI & Data Junior Consultant is expected to work closely with Service Sales Specialist and all the Sales Account Team to help drive company growth, and being recognized as a trusted advisor from the customer’s point of view.

What do we offer?

Join a dynamic team working on innovative cutting-edge AI and Data projects, with a positive work environment that is supportive for your growth in both technology and business areas. This COE will present a unique opportunity for new graduates to be exposed to innovations, projects, and customers across the globe in a fast-revolving technology space. Specific training and career development opportunities will be provided to the successful candidates. HPE offers competitive salary along with an excellent compensation and benefits package.

Responsibilities

Responsible for delivery of assigned tasks within the project lifecycle. Understand the company's technology portfolio and deliver technical design that meets customer requirements. Assignments may include developing advanced analytics and ML/DL models, implementing data engineering pipelines, integrating AI & Data product ecosystem, applying MLOps best practices and tools, and contributing to the design of AI and Data technical solutions with moderate risk/complexity.
Work under supervision of the technical lead and with customer nominated representatives to accomplish assigned tasks.
Integrate technical knowledge and business understanding to create solutions for customers.
Apply company solutions to meet moderately complex customer needs within area(s) of technical responsibility.
Provide technical consulting during project presales discussions.
Resolve technical issues independently within your technical area. Work with team members to solution cross-technology assignments or address complex technical issues.
Balance internal needs with customer needs within defined parameters.
May identify additional services that could lead to future service revenue growth.
Manage smaller projects/programs in the team.
Act professionally as trusted advisor in the consultant role for customers and the HPE sales team.
Participates as part of a team in developing and growing good relationships with team members and customers in assigned projects.
Understand the company's strategy.
Is aware of AI and Data knowledge tools and their applications.
Participate in technical or professional community events.
Keep refreshing technical knowledge and ramp up rapidly.

Education And Experience Required

Bachelor’s degree (Computer Science, Electrical & Computer Engineering, Mathematics, Physics, or related fields).
Completed or undertaking master’s degree study specialized in Artificial Intelligence & Machine Learning or Data Science.

Knowledge And Skills

Knowledge of programming languages such as Python, SQL, R, Java, C, Unix Shell scripting.
Conceptual understanding of Information Technology (IT) environments. 
Passion for new and emerging AI and Data technologies, and ability to keep updated in technological innovations and new releases.
Willingness to participate in international projects and availability to travel. 
Active listening skills and ability to adjust message to audience level.
Communicate effectively to both technical group and the management within the organization. 
Problem-solving skills (proactive, reactive, and creative). Ability to identify alternatives, set priorities, and identify solutions.
Able to produce basic documentation to required standards.
Demonstrate understanding of assigned discipline or technical area.
Proactive learn aptitude and interested in developing a career in the organization for future growth
Teamwork skills. Autonomy with the ability to organize and manage own work.

Additional Skills

Accountability, Accountability, Active Learning, Active Listening, Bias, Business Growth, Client Expectations Management, Coaching, Creativity, Critical Thinking, Cross-Functional Teamwork, Customer Centric Solutions, Customer Relationship Management (CRM), Design Thinking, Empathy, Follow-Through, Growth Mindset, Information Technology (IT) Infrastructure, Infrastructure as a Service (IaaS), Intellectual Curiosity, Long Term Planning, Managing Ambiguity, Process Improvements, Product Services, Relationship Building {+ 5 more}

What We Can Offer You

Health & Wellbeing

We strive to provide our team members and their loved ones with a comprehensive suite of benefits that supports their physical, financial and emotional wellbeing.

Personal & Professional Development

We also invest in your career because the better you are, the better we all are. We have specific programs catered to helping you reach any career goals you have — whether you want to become a knowledge expert in your field or apply your skills to another division.

Diversity, Inclusion & Belonging

We are unconditionally inclusive in the way we work and celebrate individual uniqueness. We know diverse backgrounds are valued and succeed here. We have the flexibility to manage our work and personal needs. We make bold moves, together, and are a force for good.

Let's Stay Connected"""

# Classify the job description
sections = classify_job_description(job_description_example)
sections

{'requirements': 'Is aware of AI and Data knowledge tools and their applications.\nEducation And Experience Required\n\nBachelor’s degree (Computer Science, Electrical & Computer Engineering, Mathematics, Physics, or related fields).\nCompleted or undertaking master’s degree study specialized in Artificial Intelligence & Machine Learning or Data Science.\nKnowledge And Skills\n\nKnowledge of programming languages such as Python, SQL, R, Java, C, Unix Shell scripting.\nConceptual understanding of Information Technology (IT) environments.\nPassion for new and emerging AI and Data technologies, and ability to keep updated in technological innovations and new releases.\nProblem-solving skills (proactive, reactive, and creative).\nAble to produce basic documentation to required standards.',
 'responsibilities': "About the job\nThis role has been designed as ‘Hybrid’ with an expectation that you will work on average 2-3 days per week from an HPE office.\nWho We Are\n\nHewlett Packard Enterpr

In [19]:
def classify_cv_sections(text):
    # Initialize sections
    sections = {
        'education': [],
        'experience': [],
        'first_third': '',
        'second_third': ''
    }
    
    # Predefined keywords for education and experience
    keywords_education = [
        "education", "university", "college", "school", "bachelor’s", "master’s", "phd", 
        "associate degree", "diploma", "academic", "scholarship", "graduate", "undergraduate", 
        "mba", "b.sc", "m.sc", "ph.d", "coursework", "curriculum", "credits", "major", "minor", 
        "faculty", "department", "dean's list", "honors", "cum laude", "magna cum laude", 
        "summa cum laude", "thesis", "dissertation", "capstone", "internship", "training", 
        "certifications", "certified", "license", "accredited", "BSc", "MSc", "PhD", "BA", "MA",
        "BS", "MS", "engineering", "management", "science", "technology", "informatics","relevant courses",
        "relevant coursework", "gpa", "grade point average", "academic merit", "final capstone"

    ]

    keywords_education = keywords_education + [word.capitalize() for word in keywords_education]
    
    keywords_experience = [
        "experience", "career", "professional", "internship", "position", "role", "job", 
        "employment", "work", "project", "task", "duty", "responsibility", "achievement", 
        "accomplishment", "contribution", "involvement", "engagement", "assignment", 
        "managed", "developed", "created", "implemented", "led", "coordinated", "collaborated", 
        "participated", "supported", "maintained", "analyzed", "planned", "executed", 
        "evaluated", "improved", "optimized", "delivered", "resolved", "engineered", 
        "designed", "consulted", "programmed", "configured", "deployed", "tested", 
        "upgraded", "automated", "volunteer", "consultancy", "freelance", "full-time", 
        "part-time", "seasonal", "contract", "temporary", "tenure", "apprenticeship",
        "project", "team", "teamwork", "leadership", "management", "supervision",
        "communication", "problem-solving", "decision-making", "analytical", "technical",
        "stakeholder", "client", "customer", "business", "industry", "market", "strategy",
        "planning", "development", "implementation", "execution", "monitoring", "evaluation",
        "reporting", "presentation", "training", "mentoring", "coaching", "guidance",
        "support", "feedback", "review", "assessment", "audit", "quality", "performance",
        "process", "procedure", "methodology", "technology", "solution", "system",
        "software", "hardware", "database", "network", "cloud", "security", "architecture",
        "framework", "model", "algorithm", "tool", "language", "library", "platform",
        "environment", "standard", "requirement", "regulation", "compliance", "policy"
    ]

    keywords_experience = keywords_experience + [word.capitalize() for word in keywords_experience]
    
    # Process the document with spaCy
    doc = nlp(text)
    
    # Split the CV into thirds
    one_third_length = int(len(doc) / 3)
    sections['first_third'] = doc[:one_third_length].text
    sections['second_third'] = doc[one_third_length:].text

    # Iterate over sentences to classify them based on keywords
    for sent in doc.sents:
        sentence = sent.text.strip().lower()
        if any(kw.lower() in sentence for kw in keywords_education):
            sections['education'].append(sent.text.strip())
        elif any(kw.lower() in sentence for kw in keywords_experience):
            sections['experience'].append(sent.text.strip())

    # Join sentences for education and experience sections
    for key in ['education', 'experience']:
        sections[key] = "\n".join(sections[key])

    return sections


In [20]:
def embed_text(text):
    return model.encode(text, convert_to_tensor=True)

In [21]:
def find_similarity(text1, text2):
    """Calculate cosine similarity between two pieces of text."""
    embedding1 = embed_text(text1)
    embedding2 = embed_text(text2)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [22]:
cv_text1 = """BERNARDO MARIA BEBIANO CORREIA NOBRE PALMA
Madrid, Spain | EU Passport | (+351) 914 383 855 | bernardo.n.palma@gmail.com | LinkedIn Profile EDUCATION
IE University – School of Science and Technology Madrid, Spain MSc in Business Analytics and Big Data Expected - Sep 2023 – Jul 2024
• IE Europe Scholarship | Current GPA of 3.7/4
• Relevant courses: Statistics for Data Science, Deep Learning, Computer Vision, NLP, Machine Learning, Python, SQL,
MongoDB, Spark, Tableau
• Python libraries: Pandas, NumPy, matplotlib, scikit-learn, PyTorch, pySpark.
• AI Hackathon: developed a financial statement data extractor using the OpenAI API and python.
• GenAI Bootcamp: developed a webapp MVP aiming for custom online education using the OpenAI API and python.
Instituto Superior Técnico Lisboa, Portugal BSc Degree in Industrial Engineering and Management Sep 2020 – Jul 2023
• GPA of 16/20 | Academic Merit Diploma
• Relevant courses: Elements of Programming (18/20); Microeconomics (18/20); Materials Science (18/20);
Macroeconomics (17/20); Physics II (18/20); Business Analytics (16/20).
• Final Capstone (19/20): “Uncertainty in the design and planning of sustainable supply chains”.
Developed a decision support tool using Linear Programing with python PuLp library to a multi-objective optimization approach to balance economic efficiency and environmental responsibility.
PROFESSIONAL EXPERIENCE
KOMOREBI AI (Technological Consulting startup) Madrid, Spain Intern Data Scientist April 2024 – July 2024
• Curricular Internship part of the last term of my Master’s.
• Part of the team who designed and implemented an optimization tool to manage the fleet of a leading cement company
($20BN USD in annual revenue, present in 60 countries), resulting in a 15% increase in fleet efficiency.
JUNITEC – Junior Enterprise at IST Lisbon, Portugal Head of Outsourcing Consulting Department Sep 2022 – Jan 2023
• Managed 4 ongoing outsourcing technological consulting projects simultaneously (with different themes, web
development, data science and product development), for different stakeholders including Municipality of Oeiras,
Thales (ex-Fortune 500), Calouste Gulbenkian Foundation and Quidgest, resulting in a 100K+ revenue.
• Defined a new strategical plan for the Outsourcing Department which resulted in a 40% increase in candidates and 20%
increase in project leads.
Tech-Consulting Project Manager   Sep 2021 – Sep 2022
• Lead a team of 4 members in the development of a full-stack file management platform featuring tokenized log-ins for a ONG, resulting in a 20k+ revenue.
• Lead a team of 4 members in the development of real-time responsive widgets for Thale’s (ex-Fortune 500) future applications, representing 40k+ revenue.
Member of Consulting Department
• Responsible for project leads.
• Developed 4 proposals for tech consulting projects, of which 2 were accepted.
Self-Employed Math Tutor
• Taught and counselled 4 11th-grade students in physics and math subjects.
CO-CURRICULAR ACTIVITIES Equestrian, Show Jumping
Mar 2021 – Sep 2021
Sep 2021 – Present Dec 2009 – Present
• Portuguese Champion in 2015 and 2017, and Part of the National team from 2015 to 2019.
• Participated in the FEI European Jumping Championships for Children 2016 accomplishing a 3rd in the 1st Qualifier.
• Participated in 120+ international classes in Portugal, Spain and Ireland.
• 25+ wins and 80+ top 5 finishes in national competitions.
Country Mission Head of Logistics (Volunteering) Feb 2022 – Feb 2023
• Organized logistics for “Missão País”, a Portuguese student volunteer program, mobilizing them for social and humanitarian projects during university breaks.
• Coordinated transportation, accommodation, and meal provisioning, and organized mission budget for 60+ volunteers. SKILLS
• Languages: Portuguese (Native); English (Fluent, C1, certified); Spanish (Fluent).
• Technical: Python (Advanced); Scikit-Learn (Advanced); NumPy (Advanced); SQL (Advanced); MongoDB (Advanced);
Solid Edge (Advanced); Simplex (Advanced); Tableau (Advanced); MATLAB (Intermediate); R (Advanced)."""

In [23]:
job_description = """

About Copado:

Copado is the #1 DevOps Platform for Salesforce and the Cloud. We harness the power of native CI/CD and Robotic Testing to drive digital transformation for 1,000+ of the most innovative brands on the planet — from Coca-Cola to eBay to Volkswagen. Our low-code platform unites non-technical admins and pro-code developers on the same system and empowers enterprises to scale end-to-end software delivery across multi-cloud environments.

The impact on your business? 20X faster releases, 94% fewer production bugs and 46% more Salesforce ROI. The impact on your team? No more late nights, weekend war rooms or stressful release days.

Position Summary:

We are seeking a highly motivated Junior Data Scientist to join our dynamic team. The ideal candidate will have a passion for data analysis, a strong background in Python programming, experience working with large datasets using PySpark, and proficiency in data modeling techniques. Additionally, familiarity with Salesforce data models for Marketing and Sales cloud will be highly advantageous.

What you'll be doing:

Data Analysis: Utilize Python and PySpark to analyze large datasets, extract meaningful insights, and identify patterns and trends.
Data Modeling: Apply various data modeling techniques to design, develop, and implement effective data models that address business requirements and support decision-making processes.
Salesforce Integration: Work with Salesforce data models for Marketing and Sales cloud, extract relevant data, and integrate it into the analytical pipeline for further analysis.
Algorithm Development: Develop and implement machine learning algorithms and statistical models to solve business problems and improve processes related to marketing and sales.
Data Quality: Define an implement data quality policies for data sets and models as a mechanism to prevent data-drift, feature-drift and prediction anomalies
Collaboration: Collaborate with cross-functional teams including data engineers, business analysts, and stakeholders to understand business objectives, gather requirements, and deliver actionable insights.
Documentation: Document analysis methodologies, findings, and recommendations in a clear and concise manner, and present findings to both technical and non-technical stakeholders.

Your ideal profile:

Bachelor’s degree or higher in Computer Science, Statistics, Mathematics, or a related field.
Minimum of 3 years of coding experience in Python, with a strong understanding of data structures, algorithms, and software development best practices.
At least 1 year of hands-on experience working with large datasets and using PySpark for data processing and analysis.
Proficiency in data modeling techniques such as dimensional modeling, entity-relationship modeling, and/or other relevant methodologies.
Strong communication and English language skills
Familiarity with Salesforce data models for Marketing and Sales cloud, and experience working with Salesforce APIs for data extraction and integration. Experience with Salesforce Analytics Tools including Standard Reports and Dashboards
Strong analytical and problem-solving skills, with the ability to translate business requirements into technical solutions.
Excellent communication skills and the ability to effectively collaborate with team members and stakeholders from diverse backgrounds.
Experience with machine learning libraries such as scikit-learn, TensorFlow, or PyTorch is a plus.
Knowledge of SQL for data manipulation and querying is mandatory.
Experience with data orchestration technologies such as Dagster or Airflow
Salesforce CRM Analytics or Tableau experience will be highly preferred

Benefits:

Competitive salary and benefits package.
Opportunities for professional development and career growth.
Exposure to cutting-edge technologies and projects in a collaborative environment.
Work-life balance and a supportive company culture.

Copado is Equal Employment Opportunity and Affirmative Action Employers. Qualified applicants will receive consideration for employment without regard to race, color, religion, sex, sexual orientation, gender perception or identity, national origin, age, marital status, protected veteran status, or disability status. Copado does not accept unsolicited headhunter and agency resumes. Copado will not pay any third-party agency or company that does not have a signed agreement with Copado."""
job_description2 = """ABOUT US



Indie Campers, the go-to campervan Marketplace, is on a mission of making road trips available to everyone. Thanks to its digital approach and customer-centric orientation, Indie Campers has developed a strong booking experience and high-quality road trips at affordable prices.
With over one million nights rented through our travel platform, we have welcomed more than 300,000 travelers from 169 countries. We offer a comprehensive and expanding array of road trip possibilities: short-term RV rentals, long-term campervan subscriptions, and the possibility of buying one of our vehicles available for sale.
Rooted in a deep technological focus, our challenges are both exciting and demanding and require top talent and motivation to be successfully pursued. We are growing our team and looking for those interested in pursuing this dream with us and joining an ever-developing journey.


THE ROLE



As an Analyst in the Strategy and Business Development (SBD) team at Indie Campers, you will occupy a vital and distinctive position. This role places you at the heart of the company, where you will collaborate extensively across all teams and departments. Your primary mission will be to spearhead pivotal company-wide initiatives, playing a critical role in actualizing Indie Campers vision of becoming the preeminent road trip marketplace globally.
This role is both challenging and immensely rewarding, offering unparalleled opportunities for personal and professional growth. From your very first day, you will find yourself immersed in an environment that demands responsibility and initiative. You will have the unique opportunity to work closely with, and gain insights from, the executive management team and heads of various departments. This exposure will provide you with a comprehensive understanding of the business and its operations.
As an integral member of the SBD team, your contributions will have a profound impact across an organization that is scaling at an extraordinary pace. This role is not just about analyzing and strategizing about being a key player in a dynamic and fast-growing company. You will be actively involved in shaping the future of Indie Campers, ensuring that strategic initiatives are not only conceptualized but also effectively executed to drive substantial and measurable growth.




WHAT WILL YOU WORK ON?

Strategy Development & Implementation: Take a lead role in defining and refining business strategies. This involves analyzing market trends, identifying growth opportunities, and proposing strategic directions. You'll work closely with leadership to turn these strategies into actionable plans and oversee their implementation across various business units.
New Business Ventures: Play a pivotal role in the creation of new business units, products, and projects. Your involvement will span the entire spectrum, from initial ideation through to the detailed phases of implementation and eventual handover to specialized functional or operational teams.
Analytical Insights & Market Expansion: Engage in comprehensive information gathering and analysis. Develop and apply advanced analytical frameworks to formulate and test hypotheses. Your insights will be crucial in guiding the company's expansion into new markets and the development of innovative products.
Strategic Decision Support: Actively contribute to the analysis and evaluation of key company-wide decisions, especially those affecting operations and process optimizations. Your analytical skills will help shape effective strategies and enhance operational efficiency.
Cross-Functional Project Leadership: Take on a leadership role in driving cross-functional initiatives, requiring coordination and alignment among various departments. This may include projects like international expansion, where harmonizing different functions and departments is critical.
Business Development Lifecycle: Involve yourself in the full cycle of business development activities. This includes planning, research, initial outreach, negotiation, and managing ongoing relationships with key partners to foster long-term collaborations.
Impact & Efficiency Analysis: Coordinate closely with different departments to measure and assess the impact and efficiency of new growth initiatives. These initiatives may encompass new market strategies, product launches, partnership developments, and business process improvements.
Fundraising & Investor Relations: Provide vital assistance in the structuring, evaluation, and pursuit of diverse fundraising activities and investor relations. Your role will include preparing presentations, financial models, and other materials to support these critical endeavors.
In this role, you will be essential in shaping and driving the future direction of Indie Campers. Your strategic thinking, analytical skills, and ability to execute plans will contribute significantly to our
growth and success in the market.


WHO ARE WE LOOKING FOR?

We are on the lookout for an exceptional candidate who recently graduated with a BA/BS/Masters in Business, Economics, Finance, Engineering, or Physics. Ideal candidates should have up to 2 years of professional experience, preferably in fields such as strategy consulting, management consulting, business development, or banking.
We value individuals who demonstrate a robust business acumen, characterized by the ability to dissect complex problems with a methodical and logical approach. Your analytical prowess should be top-notch, enabling you to solve intricate issues through structured thinking, sound reasoning, and intuitive judgment.
Fluency in English is a mandatory requirement, and proficiency in additional languages will be considered an advantageous asset. We're seeking ambitious, self-motivated, and entrepreneurial spirits. Candidates should be hard-working, cost-conscious, daring, tech-savvy, and have a zest for continual learning, tackling challenges, and making a significant impact.
Candidates who have gained international experience will find that it is highly appreciated in our dynamic and global environment. If you are eager to bring your analytical skills, drive, and innovative mindset to a forward-thinking team, we encourage you to apply.




OUR GLOBAL BUSINESS ACCELERATE PROGRAM:

This position is a part of our Global Business Accelerate Program
What is it?

A 4-year graduate program, that helps recent graduates to kick-start their career in an international and fast-paced environment.
This program focuses on learning by doing and developing soft and hard skills, whilst being mentored by industry experts. During this time you’ll have the opportunity to gain a better understanding of Indie Campers’ future challenges and work on how to better tackle them as well as gaining a lot of industry knowledge. This program encourages fast growth and a high level of retention, so during the first year, you’ll be doing an IEFP Professional Internship and upon performance assessment, you’ll join our team permanently. If you want to jumpstart your career by being hands-on and taking ownership of your projects this is the right program for you!


WHAT WE OFFER:

Competitive package including private health insurance, Indie Benefits, and free road trips anywhere in Europe (subject to campervan availability);
Language courses and/or other development activities are provided as a part of your annual training allowance;
A strucutured program with customized training aligned with your needs;
A dynamic position in a young, fast-growing and innovative company. At Indie Campers, no two days are the same and your contribution makes a difference;
A fun work environment with an international crowd, located in the heart of Lisbon.


Are you ready to Go Indie?"""

job_descriptions1 = [job_description, job_description2]

In [38]:
def find_top_matches(cv_text, job_descriptions, top_n=3):
    cv_sections = classify_cv_sections(cv_text)
    cv_education = cv_sections['education']
    cv_experience = cv_sections['experience']

    job_sections = [classify_job_description(job) for job in job_descriptions]
    job_requirements = [" ".join(job['requirements']) for job in job_sections]
    job_responsibilities = [" ".join(job['responsibilities']) for job in job_sections]

    # Combine all job requirements and responsibilities for comparison
    combined_job_texts = [reqs + ' ' + resp for reqs, resp in zip(job_requirements, job_responsibilities)]

    # Calculate similarities for the given pairs
    education_job_similarities = [find_similarity(cv_education, text) for text in combined_job_texts]
    experience_job_similarities = [find_similarity(cv_experience, text) for text in combined_job_texts]
    education_requirements_similarities = [find_similarity(cv_education, req) for req in job_requirements]
    experience_responsibilities_similarities = [find_similarity(cv_experience, resp) for resp in job_responsibilities]
    overall_similarities = [find_similarity(cv_text, job) for job in job_descriptions]

    # Find top N matches for overall similarity
    def get_top_matches(similarities):
        return sorted([(score, index) for index, score in enumerate(similarities)], reverse=True)[:top_n]
    
    #get a list of the rest of the comparison types base on the index of the top matches
    def get_index_functions(list1,list2):
        indexs = [element[1] for element in list1]
        top_matches = []
        for i in range(len(list2)):
            if i in indexs:
                top_matches += [list2[i]]
        return top_matches

    get_top_matches_ov = get_top_matches(overall_similarities)

    results = {
        'education_job_similarities': get_index_functions(get_top_matches_ov,education_job_similarities),
        'experience_job_similarities': get_index_functions (get_top_matches_ov,experience_job_similarities),
        'education_requirements_similarities': get_index_functions(get_top_matches_ov,education_requirements_similarities),
        'experience_responsibilities_similarities': get_index_functions(get_top_matches_ov,experience_responsibilities_similarities),
        'overall_similarities': get_top_matches_ov
    }

    return results


In [39]:
find_top_matches(cv_text1, job_descriptions1)

{'education_job_similarities': [0.19335488975048065, 0.17311739921569824],
 'experience_job_similarities': [0.08040454983711243, 0.12108443677425385],
 'education_requirements_similarities': [0.19335488975048065,
  0.17311739921569824],
 'experience_responsibilities_similarities': [0.03515321761369705,
  0.03959904983639717],
 'overall_similarities': [(0.34132814407348633, 0), (0.20246119797229767, 1)]}